# Estimating the sea ice concentration at buoy locations using the NSIDC Climate Data Record of Sea Ice Concentration

The SIC data is stored in a netCDF file. The data has coordinates "xgrid" and "ygrid" that are in the North Polar Stereographic coordinate system. So to interpolate this to buoy positions, we'll need to first find the x, y positions of the buoys. In addition, the data is at daily resolution. We'll need to pull out the buoy position at 12 UTC for each day or use a daily average position (either is fine) to line up the data.

In [82]:
import xarray as xr
import pandas as pd
import pyproj
import os
import sys
import numpy as np

# Add the path to icedrift package
sys.path.append('/Users/dwatkin2/Documents/research/packages/buoy_processing/icedrift/src/')
from icedrift.interpolation import interpolate_buoy_track

# SIC data from NSIDC
sic_data = xr.open_dataset('../data/nsidc_sic/nsidc_daily_sic_cdr_2015.nc')

# Reformat NSIDC data for convenience with interpolation -- essentially renaming coordinates and dimensions
ds = xr.Dataset({'sea_ice_concentration': (('time', 'y', 'x'), sic_data['cdr_seaice_conc'].data)},
           coords={'time': ('time', pd.to_datetime(sic_data.time.data)),
                   'x': ('x', sic_data['xgrid'].data),
                   'y': ('y', sic_data['ygrid'].data)
                 })

In [84]:
def sic_along_track(position_data, sic_data):
    """Uses the xarray advanced interpolation to get along-track sic
    via nearest neighbors."""
    # Sea ice concentration uses NSIDC NP Stereographic
    crs0 = pyproj.CRS('WGS84')
    crs1 = pyproj.CRS('+proj=stere +lat_0=90 +lat_ts=70 +lon_0=-45 +k=1 +x_0=0 +y_0=0 +a=6378273 +b=6356889.449 +units=m +no_defs')
    transformer_stere = pyproj.Transformer.from_crs(crs0, crs_to=crs1, always_xy=True)
    
    sic = pd.Series(data=np.nan, index=position_data.index)
    
    for date, group in position_data.groupby(position_data.datetime.dt.date):
        x_stere, y_stere = transformer_stere.transform(
            group.longitude, group.latitude)
        
        x = xr.DataArray(x_stere, dims="z")
        y = xr.DataArray(y_stere, dims="z")
        SIC = sic_data.sel(time=date.strftime('%Y-%m-%d'))['sea_ice_concentration'].interp(
            {'x': x,
             'y': y}, method='nearest').data

        sic.loc[group.index] = np.round(SIC.T, 3)
    sic[sic > 100] = np.nan
    return sic

In [15]:
# Load buoy data
drift_tracks_loc = "../data/buoy_data/n-ice2015/"
files = os.listdir(drift_tracks_loc)
files = [f for f in files if f.split('.')[-1] == 'csv']
files = [f for f in files if f.split('_')[0] != 'DN']
buoy_data = {file.replace('.csv', '').split('_')[-1]: 
             pd.read_csv(drift_tracks_loc + file, index_col=0,
                         parse_dates=True)
             for file in files}
buoy_data = {b: buoy_data[b] for b in buoy_data}

In [27]:
# Interpolate to hourly
buoy_data_interp = {}
for b in buoy_data:
    buoy_data_interp[b] = interpolate_buoy_track(buoy_data['2015c'], freq='1h', maxgap_minutes=240).loc[:, ['longitude', 'latitude']]

In [32]:
# We can use the resample function to pull out the data at 12 UTC
df_exmpl = buoy_data_interp[b].resample('24h', origin='12:00').asfreq().dropna()
df_exmpl.head()

,longitude,latitude
datetime,,
2015-04-21 12:00:00,17.3098,83.0976
2015-04-22 12:00:00,17.8458,83.0172
2015-04-23 12:00:00,17.6644,82.9760
2015-04-24 12:00:00,16.5700,82.8976
2015-04-25 12:00:00,16.6114,82.7752


In [44]:
daily_list = {}
for b in buoy_data_interp:
    daily_list[b] = buoy_data_interp[b].resample('24h', origin='12:00').asfreq().dropna()
    
all_positions_daily = pd.concat(daily_list)

In [45]:
# These steps make the index "flat" and change the name from the uninformative default value
all_positions_daily.reset_index(drop=False, inplace=True)
all_positions_daily.rename({'level_0': 'buoy_id'}, axis=1, inplace=True)

In [86]:
all_positions_daily['sea_ice_concentration'] = sic_along_track(all_positions_daily, ds)

In [90]:
# If desired, you can now turn it back into a dictionary with buoy ID's as keys
buoy_data_daily = {b: data for b, data in all_positions_daily.groupby('buoy_id')}

In [92]:
buoy_data_daily['2015c'].head()

,buoy_id,datetime,longitude,latitude,sea_ice_concentration
0,2015c,2015-04-21 12:00:00,17.3098,83.0976,1.0
1,2015c,2015-04-22 12:00:00,17.8458,83.0172,1.0
2,2015c,2015-04-23 12:00:00,17.6644,82.9760,1.0
3,2015c,2015-04-24 12:00:00,16.5700,82.8976,1.0
4,2015c,2015-04-25 12:00:00,16.6114,82.7752,1.0


Some things to note:
- Sea ice concentration goes from 0 to 1.
- The variable may have higher values: these are codes for different masks.
  - 2.51 = pole hole (area with no satellite coverage near the north pole)
  - 2.52 = lakes
  - 2.53 = coast
  - 2.54 = land
  - 2.55 = missing